In [1]:
using Pkg; Pkg.activate("/home/jovyan/NFMOneNotebook")

  Activating project at `~/NFMOneNotebook`


In [2]:
using GenericRofReader

using Logging
Logging.disable_logging(Logging.Warn)

Precompiling GenericRofReader
  ✓ OrderedCollections
  ✓ StaticArraysCore
  ✓ Compat
  ✓ Glob
  ✓ TickTock
  ✓ Statistics
  ✓ Compat → CompatLinearAlgebraExt
  ✓ URIs
  ✓ Crayons
  ✓ ChainRulesCore
  ✓ BenchmarkTools
  ✓ ChainRulesCore → ChainRulesCoreSparseArraysExt
  ✓ Tokenize
  ✓ DataStructures
  ✓ CSTParser
  ✓ StaticArrays
  ✓ CommonMark
  ✓ StaticArrays → StaticArraysStatisticsExt
  ✓ StaticArrays → StaticArraysChainRulesCoreExt
  ✓ JuliaFormatter
  ✓ GenericRofReader
  21 dependencies successfully precompiled in 145 seconds. 7 already precompiled.


LogLevel(1001)

In [3]:
base_path = "/home/jovyan/"
data_path = joinpath(base_path, "data/ReadOnly/dyperexprod/ford-dat-3/")

total_folder_count = length(readdir(data_path))

search_string = "USB13"
seq_paths = []

println("Start checking for matching measurements.")

folders_processed = 0
for folder in readdir(data_path, join=true)
    folders_processed += 1
    try
        rof_folders = filter(endswith(".rof"), readdir(folder))

        for rof_folder in rof_folders
            if occursin(search_string, rof_folder)
                rof_folder_path = joinpath(folder, rof_folder)
                push!(seq_paths, rof_folder_path)

                println(rof_folder_path)
            end
        end

    catch e
        println("Unexpected folder: ", folder)
        println("Catched error: ", e)
    end

    if folders_processed % 100 == 0
        processed = folders_processed / total_folder_count
        println("Processed ", round(processed * 100, digits=1), " % of all folders.")
    end    
end

println("Found ", length(seq_paths), " measurements.")

Start checking for matching measurements.
/home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/000163c9d6fb5b87a9ea825792e3419daa8d05c0e6078de200de8216451ead94/FD3_USB13_20240730_032848_001.rof
/home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/0140b320dc092651e208f3607517804932ca408912238eae3b5f4d1e812d1515/FD3_USB13_20240729_180020_001.rof
/home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/025615d5165fce0b750d99c6e0725fa3afa1a99ccb9c0d8aa3ed09b6a50b69b0/FD3_USB13_20240717_190239_001.rof
/home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/035a6dc1d241bc2e19fa7c9939bb77e28a87b2169eb42a6ae423c4f4cbc94313/FD3_USB13_20240722_180202_001.rof
Processed 1.2 % of all folders.
/home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/03b5bfc0c2431490b71cf845afc706869e8458d8f4f26e6dc4525229c91b6034/FD3_USB13_20240717_194604_001.rof
/home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/04368795b5677c919017e982831f34c67ba181b5f8769937cedd1902f7c82bf5/FD3_USB13_20240711_171949_001.rof
/home/jovyan/data/ReadOnly/dyper

In [4]:
using Statistics

desired_interface = "ValMessage.rbdyapumwrouter_gw_instance__valin_viper2_out.ValMessage"
outfolder = joinpath(base_path, "mlc2lr", search_string)
mkpath(outfolder)
outfile_path = joinpath(outfolder, "velocity_stats.csv")
outfile = open(outfile_path, "w")

velocity_statistics = []

println("Processing ", length(seq_paths), " sequences.")
for path in seq_paths
    try
        idx = 100000
        namespace_name = join("TestNamespace", string(idx))
        reader = initialize_reader(path, desired_interface, namespace_name)
        all_valid_interface_samples = read_all_valid_interfaces(reader)

        if length(all_valid_interface_samples) == 0
            println("Interface NOT found in: ", path)
            continue
        end

        parts = split(path, '/')        

        velocities_mph = [2.237136 * sample.m_pfOdometry.m_vehicleVelocity.m_value for sample in all_valid_interface_samples]

        current_stats = Dict()
        current_stats["seq"] = split(parts[9], '.')[1]
        current_stats["min"] = minimum(velocities_mph)
        current_stats["max"] = maximum(velocities_mph)
        current_stats["median"] = median(velocities_mph)
        current_stats["avg"] = mean(velocities_mph)
        current_stats["var"] = var(velocities_mph)

        velocity_sample_count = length(velocities_mph)

        current_stats["velo_samples"] = velocity_sample_count

        standstill = filter(v -> abs(v) <= 1.242854, velocities_mph) #  <= 2 km/h
        standstill_rate = length(standstill) / velocity_sample_count * 100
        current_stats["standstill"] = standstill_rate

        very_low_velo = filter(v -> 1.242854 < v <= 5, velocities_mph)
        current_stats["very_slow"] = length(very_low_velo) / velocity_sample_count * 100
        
        low_velo = filter(v -> 5 < v <= 10, velocities_mph)
        current_stats["slow"] = length(low_velo) / velocity_sample_count * 100
        
        mid_velo = filter(v -> 10 < v <= 15, velocities_mph)
        current_stats["mid"] = length(mid_velo) / velocity_sample_count * 100
        
        high_velo = filter(v -> 15 < v <= 21, velocities_mph)
        current_stats["high"] = length(high_velo) / velocity_sample_count * 100
        
        too_high_velo = filter(v -> 21 < v, velocities_mph)
        current_stats["very_high"] = length(too_high_velo) / velocity_sample_count * 100

        push!(velocity_statistics, current_stats)

    catch
        println("Interface NOT found in: ", path)
    end
end

keys = ["seq", "min", "max", "median", "avg", "var", "velo_samples", "standstill", "very_slow", "slow", "mid", "high", "very_high"]

for key in keys
    write(outfile, key, ";")
end
write(outfile, "\n")

for stats in velocity_statistics
    for key in keys
        write(outfile, string(stats[key]), ";")        
    end
    write(outfile, "\n")
end

println("output written to ", outfile_path)

close(outfile)

Processing 390 sequences.
Interface NOT found in: /home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/0f52894d171f9668ee5dbba4e6d38c9c384f93529964ba861cc27278775ba75d/FD3_USB13_158_TS158.rof
Interface NOT found in: /home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/12c7ea67d8185156c6052d3b0919c844ab9a47f05043f8291dd2e42ce8acef09/FD3_USB13_20240411_182459_001.rof
Interface NOT found in: /home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/1ededfc0013c16d5537ebf4a73eb4df758f58a87b90b35f1a2dbf91fcb77404d/FD3_USB13_194_TS194.rof
Interface NOT found in: /home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/2092598debe31406efd08a0b835b5efca237bef354da8afe41a587d93ce63e53/FD3_USB13_20240709_191049_001.rof
Interface NOT found in: /home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/35aa66bba464153a94c11e13544d8c7af1ed6a6c1a7b24f110efebb821ab2db8/FD3_USB13_182_TS182.rof
Interface NOT found in: /home/jovyan/data/ReadOnly/dyperexprod/ford-dat-3/38a0133ee5f71fb90e6edb1a95a42420e85d9b9c2586b4fe297acf0819ac85ef/